In [5]:
# ✅ 掛載 Google Drive
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# prompt: pd

import pandas as pd


In [7]:
# 📥 載入兩份 CSV：clean + BERTopic
df_clean = pd.read_csv("1.2_jay_comments_clean.csv")
df_topic = pd.read_csv("2.1_BerTopic_jay_bertopic_topic_result.csv")

# 🔁 合併資料（用 text_cleaned 對 text）
df = pd.merge(df_clean, df_topic, left_on="text_cleaned", right_on="text", how="inner")

# 🧹 移除重複欄位，保留 _x 欄為主
columns_to_keep = [col for col in df.columns if not col.endswith("_y")]
df = df[columns_to_keep]

# 🪄 移除所有 _x 後綴，變回正常欄名
df.columns = [col.replace("_x", "") for col in df.columns]

# 查看欄位總覽（確認有你要的）
print(df.columns.tolist())

# 顯示前 5 筆資料（重點欄位檢查）
df[["text_cleaned", "bertopic_topic", "bertopic_prob"]].head()

['video_id', 'author', 'text', 'published_at', 'video_title', 'text_cleaned', 'bertopic_topic', 'bertopic_prob']


,text_cleaned,bertopic_topic,bertopic_prob
0,徐若瑄那時期很正很漂亮,202,0.129156
1,初中時是用這首歌追到老師的 老師的孩子現在也應該是上初中了,2,0.090893
2,徐若瑄那個時候真的美到驚心動魄,202,1.000000
3,所以才會有今生一見如故的感覺,-1,0.000649
4,2025年還在聽,533,1.000000


In [8]:
pip install --upgrade openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.84.0
    Uninstalling openai-1.84.0:
      Successfully uninstalled openai-1.84.0


In [ ]:
# ✅ 載入套件
import openai
import pandas as pd
import json
from tqdm import tqdm
from tenacity import retry, stop_after_attempt, wait_random_exponential
import os

# ✅ 設定檔案儲存路徑（可修改成你要的資料夾）
checkpoint_path = "/content/drive/MyDrive/absa_jay_comments_checkpoint.csv"
final_output_path = "/content/drive/MyDrive/absa_jay_comments_output.csv"

# ✅ 初始化 OpenAI client（新版 SDK）
client = openai.OpenAI(
    api_key="sk-..."
)

# ✅ 載入 CSV：clean + BERTopic
df_clean = pd.read_csv("1.2_jay_comments_clean.csv")
df_topic = pd.read_csv("2.1_BerTopic_jay_bertopic_topic_result.csv")

# 🔁 合併資料（用 text_cleaned 對 text）
df = pd.merge(df_clean, df_topic, left_on="text_cleaned", right_on="text", how="inner")
columns_to_keep = [col for col in df.columns if not col.endswith("_y")]
df = df[columns_to_keep]
df.columns = [col.replace("_x", "") for col in df.columns]

# ✅ 限制只處理前 1000 筆
df = df.head(1000)

# ✅ Prompt 模板
def build_prompt(text):
    return f"""
請閱讀以下粉絲留言，並判斷留言主要是在評論哪一個面向，情緒是正面、中立還是負面。
面向選項如下：
- 音樂風格（如旋律、節奏、演唱）
- 歌詞意境（如故事內容、隱喻、敘事）
- 情感共鳴（如感動、哭了、有共鳴）
- 對比分析（與早期作品、其他藝人比較）
- 青春回憶（如提到聽歌時的學生時代、重要人生經歷）

請回傳標準 JSON 格式，例如：
{{ "aspect": "情感共鳴", "sentiment": "正面" }}

粉絲留言：{text}
""".strip()

# ✅ GPT ABSA 函數（含 retry/backoff 機制）
@retry(stop=stop_after_attempt(3), wait=wait_random_exponential(min=1, max=10))
def gpt_absa(text):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "你是一個中文評論分析助手"},
                {"role": "user", "content": build_prompt(text)}
            ],
            temperature=0.3,
        )
        content = response.choices[0].message.content.strip()
        print("✅ 回傳：", content)
        json_start = content.find("{")
        json_end = content.rfind("}") + 1
        json_str = content[json_start:json_end]
        parsed = json.loads(json_str)
        return parsed.get("aspect", "無法判斷"), parsed.get("sentiment", "無法判斷")
    except Exception as e:
        print("❌ 錯誤：", e)
        return "無法判斷", "無法判斷"

# ✅ 檢查是否有 checkpoint
if os.path.exists(checkpoint_path):
    print(f"🔁 偵測到現有 checkpoint，載入進度：{checkpoint_path}")
    df_checkpoint = pd.read_csv(checkpoint_path)
    aspects = df_checkpoint["aspect"].tolist()
    sentiments = df_checkpoint["sentiment"].tolist()
    start_idx = len(aspects)
else:
    print("🚀 尚無 checkpoint，從頭開始")
    aspects = []
    sentiments = []
    start_idx = 0

# ✅ 主迴圈（僅處理前 1000 筆）
for idx, text in tqdm(enumerate(df["text_cleaned"]), total=len(df), desc="GPT ABSA 處理中"):
    if idx < start_idx:
        continue

    aspect, sentiment = gpt_absa(text)
    aspects.append(aspect)
    sentiments.append(sentiment)

    if (idx + 1) % 100 == 0:
        df_partial = df.iloc[:idx + 1].copy()
        df_partial["aspect"] = aspects
        df_partial["sentiment"] = sentiments
        df_partial.to_csv(checkpoint_path, index=False)
        print(f"✅ 存檔 checkpoint at row {idx+1}")

# ✅ 最終儲存
df["aspect"] = aspects
df["sentiment"] = sentiments
df.to_csv(final_output_path, index=False)
print(f"🎉 全部完成！結果已儲存至：{final_output_path}")

🚀 尚無 checkpoint，從頭開始


GPT ABSA 處理中:   0%|          | 1/1000 [00:01<22:49,  1.37s/it]

✅ 回傳： {
    "aspect": "青春回憶",
    "sentiment": "正面"
}


GPT ABSA 處理中:   0%|          | 2/1000 [00:02<16:08,  1.03it/s]

✅ 回傳： {
    "aspect": "青春回憶",
    "sentiment": "正面"
}


...

✅ 回傳： { "aspect": "情感共鳴", "sentiment": "正面" }


GPT ABSA 處理中: 100%|█████████▉| 999/1000 [1:39:36<00:09,  9.64s/it]

✅ 回傳： {
  "aspect": "對比分析",
  "sentiment": "正面"
}


GPT ABSA 處理中: 100%|██████████| 1000/1000 [1:39:45<00:00,  5.99s/it]

✅ 回傳： {
    "aspect": "情感共鳴",
    "sentiment": "正面"
}
✅ 存檔 checkpoint at row 1000
🎉 全部完成！結果已儲存至：/content/drive/MyDrive/absa_jay_comments_output.csv
